<a href="https://colab.research.google.com/github/mmillet88/russian_therapy/blob/main/pre_processingResponseDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading libraries

In [ ]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.4 MB/s 
     |████████████████████████████████| 365 kB 71.6 MB/s 
     |████████████████████████████████| 6.6 MB 47.1 MB/s 
     |████████████████████████████████| 596 kB 59.7 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 212 kB 28.8 MB/s 
     |████████████████████████████████| 141 kB 45.5 MB/s 
     |████████████████████████████████| 127 kB 54.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account al

In [ ]:
import os
import pandas as pd 
import matplotlib.pyplot as plt

from transformers import AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import EarlyStoppingCallback
from transformers import (
    glue_tasks_num_labels,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    GlueDataset,
    GlueDataTrainingArguments,
    TrainingArguments,
)
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
import gc

import datasets
from datasets import load_dataset
from datasets import DatasetDict

import numpy as np
from numpy.lib.function_base import average

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=84b3707562ffd442bdf8af24d6c5735c34df47553ebeee245f85c6db1edd84f3
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Downloading a dataset

In [ ]:
import os
!mkdir /content/emp_chatbot
!mkdir emp_chatbot/data

mkdir: cannot create directory ‘/content/emp_chatbot’: File exists
mkdir: cannot create directory ‘emp_chatbot/data’: No such file or directory


In [ ]:
!ls

all_emotions_responses.json


In [ ]:
os.chdir('/content/emp_chatbot/data')

In [ ]:
# https://drive.google.com/file/d/121M9d19m7ljXaw0f5pggpHd9gsN2Hu3U/view?usp=sharing
# !gdown --id 1GaaG8UYKIHWSXR1X3OuxEfBpD2i-3qar
!gdown --id 1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
# https://drive.google.com/file/d/1GaaG8UYKIHWSXR1X3OuxEfBpD2i-3qar/view?usp=sharing
# !gdown --id 1UhqyvObGgMl3_jEIkDiYJQEkQ7rFOXOc

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1uRPD0QPf-I90CopU9eccvzuqxSfsLcPp
To: /content/emp_chatbot/data/all_emotions_responses.json
100% 2.38M/2.38M [00:00<00:00, 245MB/s]


In [ ]:
# data_files_EPru = {"train": "sadness_questions_follow_up.json"}
data_files_EPru = {"train": "all_emotions_responses.json"}
epru = load_dataset('json', data_files=data_files_EPru)

Using custom data configuration default-3a1f74a7752a7548


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3a1f74a7752a7548/0.0.0/a3e658c4731e59120d44081ac10bf85dc7e1388126b92338344ce9661907f253. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

# Pre-computing Fluency and Embeddings




## Fluency/perplexity

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, AutoModelWithLMHead, AutoTokenizer

device = "cuda"
model_id = "sberbank-ai/ruT5-base"
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
model_LM = AutoModelWithLMHead.from_pretrained(model_id).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
columns = epru['train'].column_names
print(columns)

['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?']


In [ ]:
columns = epru['train'].column_names
columns_indices = list(range(4,len(columns)))
perplexity_columns = [columns[index] for index in columns_indices]

In [ ]:
perplexity_columns

['Was this emotion triggered by a specific event?',
 'Was the event recent?',
 'Did you find protocol 6 distressing?',
 'Did you find protocol 11 distressing?',
 'Is it ok to ask additional questions?',
 'Have you strongly felt or expressed any of the following emotions towards someone?',
 'Do you believe you should be the savior of someone else?',
 'Do you feel that you are trying to control someone?',
 'Do you see yourself as the victim, blaming someone else for how negative you feel?',
 'Are you always blaming and accusing yourself for when something goes wrong?',
 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?',
 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?',
 'Would you like to attempt protocols for joy?']

In [ ]:
def calculate_perplexity(row, column_name):
  if len(row[column_name]) != 0:
    encodings = tokenizer(row[column_name], return_tensors="pt")
    input_ids = encodings.input_ids.to(device)
    target_ids = input_ids.clone()
    with torch.no_grad():
      outputs = model_LM(input_ids, labels=target_ids)
      neg_log_likelihood = outputs[0]
    string_name = f"ppl_{column_name}"
    ppl = torch.exp(neg_log_likelihood)
    return {f'ppl_{column_name}': 1/ppl.item()}
  else:
    return {f'ppl_{column_name}': 0}

In [ ]:
for column_name in perplexity_columns:
  epru = epru.map(lambda x: calculate_perplexity(x, column_name))

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

...

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

In [ ]:
# change dataset into pandas 
epru.set_format(type="pandas")
df = epru['train'][:]
df.head()
# emotions = set(df['emotion'].tolist())

,gender,age,emotion,How are you feeling?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,Did you find protocol 11 distressing?,Is it ok to ask additional questions?,Have you strongly felt or expressed any of the following emotions towards someone?,...,ppl_Did you find protocol 11 distressing?,ppl_Is it ok to ask additional questions?,ppl_Have you strongly felt or expressed any of the following emotions towards someone?,ppl_Do you believe you should be the savior of someone else?,ppl_Do you feel that you are trying to control someone?,"ppl_Do you see yourself as the victim, blaming someone else for how negative you feel?",ppl_Are you always blaming and accusing yourself for when something goes wrong?,ppl_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,ppl_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,ppl_Would you like to attempt protocols for joy?
0,male,30-39,fear,я переживаю - смогу ли выполнить поставленную ...,"Я очень сопереживаю вам сейчас. Скажите, пожал...","понимаю вас, сочувствую.Это чувство появилось ...","Скажите, Методика 6 по ощущениям Вам была непр...","Скажите, Методика одиннадцать по ощущениям Вам...","позвольте узнать у вас подробнее, что было в д...",как вы считаете испытываете ли вы к кому нибу...,...,0.001215,0.003141,0.000854,0.004831,0.000583,0.000083,0.002533,0.004711,0.002785,0.0
1,male,30-39,fear,В последние дни испытываю стресс. Это связано ...,"Спасибо, что уточнили Ваше ощущение. Ваша трев...","Спасибо за доверие. Скажите, ваши переживания ...",Вызвала ли внутренний дискомфорт методика 6? С...,Вызвала ли внутренний дискомфорт методика один...,Мне нужно лучше узнать ситуацию. Вы не возража...,"Вопрос сложный, и отвечать на него обычно труд...",...,0.000099,0.005612,0.002863,0.001305,0.006710,0.004760,0.001445,0.003490,0.000972,0.0
2,male,25-29,fear,Чувствую себя не очень хорошо. Те ситуации и в...,"Как вы считаете, ваши ощущения, мысли, чувства...","Я вас понимаю, мне искренне жаль, что так прои...","Поделитесь со мной, пожалуйста, 6 методика выз...","Поделитесь со мной, пожалуйста, одиннадцать ме...",Мне бы хотелось задать вам несколько уточняющи...,Сейчас я вынужден задать такой вопрос. Для мно...,...,0.000732,0.004303,0.001029,0.000774,0.017881,0.001645,0.003906,0.000004,0.003688,0.0
3,male,25-29,fear,"Я переживаю из-за того, что мне кажется будто ...",Ваша тревога связана с каким-то конкретным соб...,"Сожалею, что вам плохо. Ваше чувства тревоги и...",Я тебя так понимаю и очень хочу помочь. Можно ...,Я тебя так понимаю и очень хочу помочь. Можно ...,Чтобы прояснить ситуацию мне бы хотелось задат...,"Это трудный вопрос, я понимаю. Вы чувствовали ...",...,0.001282,0.000201,0.003062,0.018126,0.001279,0.003345,0.007838,0.007593,0.000606,0.0
4,male,18-24,fear,"Я не могу спать , в голове полно мыслей ,котор...","Задумайтесь, что могло повлиять на Ваше чувств...","Я сожалею ,что Вы испытываете чувство тревоги....",Я вам буду благодарен если Вы ответите мне - М...,Я вам буду благодарен если Вы ответите мне - М...,Вы не против ответить на несколько моих вопрос...,Испытываете ли к кому-нибудь негативные чувств...,...,0.001799,0.001572,0.000018,0.001456,0.005590,0.012699,0.000228,0.006949,0.003146,0.0


In [ ]:
df[df['emotion']=='fear']['ppl_Did you find protocol 6 distressing?'].max()

0.05498476688882713

In [ ]:
len(df.emotion.unique())
max_ppl = dict()
for emotion_type in df.emotion.unique():
  max_ppl.setdefault(emotion_type,[])
  for column in perplexity_columns:
    val = df[df['emotion']==emotion_type][f"ppl_{column}"].max()
    lst = max_ppl.get(emotion_type)
    lst.append(val)
    max_ppl[emotion_type] = lst

In [ ]:
max_ppl['fear']


[0.021503003722159575,
 0.022479849567217616,
 0.05498476688882713,
 0.06302268676322594,
 0.021175940672727735,
 0.06271603239943957,
 0.018126051787968674,
 0.08631315055313789,
 0.04535789042483659,
 0.014785702521664385,
 0.009952087659037734,
 0.010632120063657094,
 0.0]

In [ ]:
def normalise_ppl(row, column, largest_ppl):
  row[column] = row[column]/largest_ppl
  return row


In [ ]:
epru.reset_format()

In [ ]:
normalised_epru = epru
for index, column in enumerate(perplexity_columns):
  normalised_epru = normalised_epru.map(lambda x: normalise_ppl(x, f"ppl_{column}", largest_ppl= max_ppl[x['emotion']][index]))

  0%|          | 0/492 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

...

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

In [ ]:
# epru[train]["ppl_Did you find protocol 6 distressing?"]
normalised_epru['train']["ppl_Have you strongly felt or expressed any of the following emotions towards someone?"]

[0.013614003317762026,
 0.0456441676060807,
 0.016401661055693042,
 0.04881650075492994,
 0.00028321368489731496,
 0.8897985452357846,
 0.04374138694099306,
 0.06482581866658198,
 0.11449964481974835,
 0.01090253998584061,
 0.03459802900666074,
 0.0515423156494856,
 0.15651179416837968,
 0.28050796841156905,
 0.024060805669593115,
 0.017997352188432732,
 0.044685239709704436,
 0.10958012407203678,
 0.09673810682105798,
 0.03739810143172014,
 0.030221289409482884,
 0.03103010000944298,
 0.02398869068804594,
 0.015509539934416585,
 0.07146989092225252,
 0.022102261363300832,
 0.08623119125762245,
 0.05242147707532478,
 0.09261667798342539,
 0.17437079388997076,
 0.10747028087423963,
 0.007731895454933442,
 0.015991850555807083,
 0.023398803178351947,
 0.036126107562564104,
 0.06359389444145906,
 0.08280037343856501,
 0.027438029211136625,
 0.03264018998618258,
 0.04681943780959578,
 0.17443691247063997,
 0.10552945519277311,
 0.059276653742209966,
 0.009728793819156416,
 5.51306665378623

## Empathy

In [ ]:
model_id = "mmillet/distilrubert-tiny-cased-conversational-v1_single_finetuned_empathy_classifier"
classifier = pipeline('text-classification', model=model_id)

In [ ]:
preds = classifier(epru['train'][8]["Have you strongly felt or expressed any of the following emotions towards someone?"], top_k=3)
preds = classifier("Это очень хорошо, что вы открыты, очень постараюсь вам помочь. Уверен, что у нас все получится. Задам вам дополнительные вопросы.", top_k=3)
print(preds)
prediction = max(preds, key=lambda x: x['score'])
print(prediction)

[{'label': 'LABEL_0', 'score': 0.8753650784492493}, {'label': 'LABEL_2', 'score': 0.11078225821256638}, {'label': 'LABEL_1', 'score': 0.01385271642357111}]
{'label': 'LABEL_0', 'score': 0.8753650784492493}


In [ ]:
epru['train'][8]["Have you strongly felt or expressed any of the following emotions towards someone?"]

'Есть не самые приятные вопросы, но они имеют значение. Пожалуйста, вспомните, сталкивались ли вы с завистью, ненавистью или злостью, направленной на другого человека с вашей стороны?'

In [ ]:
pipeline('text-classification', model=self.model_id)

preds = self.classifier(text, top_k=3)
prediction = max(preds, key=lambda x: x['score'])
prediction_label = self.int2label(int(prediction['label'][-1]))

NameError: ignored

In [ ]:
def calculate_empathy(row, column_name):
  if len(row[column_name]) != 0:
    # print(type(row[column_name]))
    preds = classifier(row[column_name], top_k=3)
    prediction = max(preds, key=lambda x: x['score'])
    if prediction["label"] == 'LABEL_0':
      score = 1
    elif prediction["label"] == 'LABEL_1':
      score = 0.33
    else:
      score = 0.67
    return {f'emp_{column_name}': score}
  else:
    return {f'emp_{column_name}': 0}

In [ ]:
for column_name in perplexity_columns:
  normalised_epru = normalised_epru.map(lambda x: calculate_empathy(x, column_name))

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

...

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

  0%|          | 0/492 [00:00<?, ?ex/s]

In [ ]:
# change dataset into pandas 
normalised_epru.set_format(type="pandas")
df = normalised_epru['train'][:]
# df["emp_Have you strongly felt or expressed any of the following emotions towards someone?"]
df.head()

,gender,age,emotion,How are you feeling?,Was this emotion triggered by a specific event?,Was the event recent?,Did you find protocol 6 distressing?,Did you find protocol 11 distressing?,Is it ok to ask additional questions?,Have you strongly felt or expressed any of the following emotions towards someone?,...,emp_Did you find protocol 11 distressing?,emp_Is it ok to ask additional questions?,emp_Have you strongly felt or expressed any of the following emotions towards someone?,emp_Do you believe you should be the savior of someone else?,emp_Do you feel that you are trying to control someone?,"emp_Do you see yourself as the victim, blaming someone else for how negative you feel?",emp_Are you always blaming and accusing yourself for when something goes wrong?,emp_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?,emp_Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?,emp_Would you like to attempt protocols for joy?
0,male,30-39,fear,я переживаю - смогу ли выполнить поставленную ...,"Я очень сопереживаю вам сейчас. Скажите, пожал...","понимаю вас, сочувствую.Это чувство появилось ...","Скажите, Методика 6 по ощущениям Вам была непр...","Скажите, Методика одиннадцать по ощущениям Вам...","позвольте узнать у вас подробнее, что было в д...",как вы считаете испытываете ли вы к кому нибу...,...,1.00,0.33,0.33,0.33,0.33,0.33,0.33,0.33,0.67,0.0
1,male,30-39,fear,В последние дни испытываю стресс. Это связано ...,"Спасибо, что уточнили Ваше ощущение. Ваша трев...","Спасибо за доверие. Скажите, ваши переживания ...",Вызвала ли внутренний дискомфорт методика 6? С...,Вызвала ли внутренний дискомфорт методика один...,Мне нужно лучше узнать ситуацию. Вы не возража...,"Вопрос сложный, и отвечать на него обычно труд...",...,1.00,0.67,0.33,1.00,0.33,0.33,0.33,1.00,1.00,0.0
2,male,25-29,fear,Чувствую себя не очень хорошо. Те ситуации и в...,"Как вы считаете, ваши ощущения, мысли, чувства...","Я вас понимаю, мне искренне жаль, что так прои...","Поделитесь со мной, пожалуйста, 6 методика выз...","Поделитесь со мной, пожалуйста, одиннадцать ме...",Мне бы хотелось задать вам несколько уточняющи...,Сейчас я вынужден задать такой вопрос. Для мно...,...,0.33,1.00,1.00,1.00,0.33,0.33,0.33,0.67,1.00,0.0
3,male,25-29,fear,"Я переживаю из-за того, что мне кажется будто ...",Ваша тревога связана с каким-то конкретным соб...,"Сожалею, что вам плохо. Ваше чувства тревоги и...",Я тебя так понимаю и очень хочу помочь. Можно ...,Я тебя так понимаю и очень хочу помочь. Можно ...,Чтобы прояснить ситуацию мне бы хотелось задат...,"Это трудный вопрос, я понимаю. Вы чувствовали ...",...,1.00,0.67,0.67,1.00,0.33,0.33,0.33,0.33,1.00,0.0
4,male,18-24,fear,"Я не могу спать , в голове полно мыслей ,котор...","Задумайтесь, что могло повлиять на Ваше чувств...","Я сожалею ,что Вы испытываете чувство тревоги....",Я вам буду благодарен если Вы ответите мне - М...,Я вам буду благодарен если Вы ответите мне - М...,Вы не против ответить на несколько моих вопрос...,Испытываете ли к кому-нибудь негативные чувств...,...,1.00,1.00,0.33,1.00,0.33,0.33,0.33,0.33,1.00,0.0


In [ ]:
normalised_epru.reset_format()

## Generate embeddings of emotional utterances

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
pipe = pipeline("feature-extraction", model=model, tokenizer=tokenizer)
features = pipe("Чувствую себя очень плохо. Мне очень грустно.")

In [ ]:
def generate_embeddings(row):
  embedding = pipe(row['How are you feeling?'])
  return {'embeddings': embedding}

In [ ]:
normalised_epru_with_embeddings= normalised_epru.map(generate_embeddings)

  0%|          | 0/492 [00:00<?, ?ex/s]

# Save dataset

In [ ]:
normalised_epru_with_embeddings.save_to_disk("/content/drive/MyDrive/chatbot/ResponseGenerationDataset")

# Checking dataset

In [ ]:
from datasets.load import load_from_disk
ds = load_from_disk("/content/drive/MyDrive/chatbot/ResponseGenerationDataset")

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['gender', 'age', 'emotion', 'How are you feeling?', 'Was this emotion triggered by a specific event?', 'Was the event recent?', 'Did you find protocol 6 distressing?', 'Did you find protocol 11 distressing?', 'Is it ok to ask additional questions?', 'Have you strongly felt or expressed any of the following emotions towards someone?', 'Do you believe you should be the savior of someone else?', 'Do you feel that you are trying to control someone?', 'Do you see yourself as the victim, blaming someone else for how negative you feel?', 'Are you always blaming and accusing yourself for when something goes wrong?', 'Is it possible that in previous conversations you may not have always considerdother viewpoints presented?', 'Are you undergoing a personal crisis (experience difficulties with loved ones e.g. falling out with friends)?', 'Would you like to attempt protocols for joy?', 'ppl_Was this emotion triggered by a specific event?', 'ppl

In [ ]:
ds['train']["ppl_Is it possible that in previous conversations you may not have always considerdother viewpoints presented?"]

[0.4733383294479988,
 0.3507293405182644,
 0.00041879916473704716,
 0.7630020504992654,
 0.6981990588259245,
 0.02821486173843643,
 0.006279223608939661,
 0.336825100582313,
 0.41712205934107754,
 0.18007022933393538,
 0.05609696952804117,
 0.08966296701829175,
 0.043630802849849666,
 0.35174999220131803,
 0.3469848681182123,
 0.18912806522605988,
 1.9703844539057774e-05,
 0.34798186244922674,
 0.22616289193370848,
 0.4024413727867989,
 0.14679141607557636,
 0.30675321396396193,
 0.3070237783414681,
 0.14192162692032692,
 0.07666940333873683,
 0.08577325400919875,
 0.1645854890163887,
 0.08551572866352929,
 0.4569984075167817,
 0.09008745766191786,
 0.49387211382992463,
 0.09605942495084964,
 0.3566451514648329,
 0.2227012566150226,
 0.06946218786718654,
 0.4784134763479545,
 0.3263551431697343,
 0.32571246298166134,
 0.4506635038136824,
 0.2004549324633048,
 0.6445476240767324,
 0.05915657150208072,
 0.21977204171254464,
 0.0,
 0.059232445344952867,
 0.36657976949146504,
 0.5323576281

In [ ]:
util.cos_sim(features[0][0], features[0][0])

tensor([[1.]])

In [ ]:
querry = "Чувствую себя очень плохо. Мне очень грустно."
querry_embedding = pipe(querry)

In [ ]:
def calculate_similarity(row, querry_embedding):
  simil = util.cos_sim(row['embeddings'][0][0], querry_embedding)
  return {'similarity': simil}

In [ ]:
sad_dialogue= sad_dialogue.map(lambda x: calculate_similarity(x, querry_embedding[0][0]))#, remove_columns=['embeddings'])


  0%|          | 0/92 [00:00<?, ?ex/s]

In [ ]:
sorted_dataset = sad_dialogue.sort('similarity')
top_10_similar = sorted_dataset['train'].select([0,1,2,3,4,5,6,7,8,9,10])

In [ ]:
top_10_similar

Dataset({
    features: ['age', 'sad_utterance', 'how_recent_is_the_event?', 'specific_event_trigger', 'additional_questions_permission', 'feeling_envy_mistrust_trigger', 'method_6', 'self_blame_trigger', 'gender', 'ppl_how_recent_is_the_event?', 'ppl_specific_event_trigger', 'ppl_additional_questions_permission', 'ppl_feeling_envy_mistrust_trigger', 'ppl_method_6', 'embeddings', 'similarity'],
    num_rows: 11
})

# Novelty


In [ ]:
import nltk
from nltk.util import ngrams
 
# Function to generate n-grams from sentences.
def extract_ngrams(data, num):
    n_grams = ngrams(nltk.word_tokenize(data), num)
    return [ ' '.join(grams) for grams in n_grams]


In [ ]:
past_sentences = dict()

In [ ]:
def generate_N_grams(text,ngram=1):
  words=[word for word in text.split(" ")] 
  # print("Sentence after removing stopwords:",words)
  temp=zip(*[words[i:] for i in range(0,ngram)])
  ans=[' '.join(ngram) for ngram in temp]
  return ans

In [ ]:
def calculate_simil(n_split_1, n_split_2, n_gram):
  candidate_response = generate_N_grams(n_split_1, n_gram)
  past_response = generate_N_grams(n_split_2, n_gram)
  common_n_grams = set(candidate_response).intersection(set(past_response))
  score = len(common_n_grams)/len(candidate_response)
  return score

In [ ]:
def generate_n_gram(sentence, max_n_gram):
  n_gram_list = [generate_N_grams(sentence, n_gram) for n_gram in range(1, max_n_gram+1)]
  return n_gram_list

In [ ]:
def novelty_score(sent_1_n_grams, sent_2_n_grams):
  common_n_grams = [set(sent_1_n_grams[i]).intersection(set(sent_2_n_grams[i])) for i in range(len(sent_1_n_grams))]
  score = [len(common_n_grams[i])/min(len(sent_1_n_grams[i]), len(sent_2_n_grams[i])) for i in range(len(common_n_grams))]
  return sum(score)/len(score)

In [ ]:
def calculate_novelty(sentence):
  new_sentence_n_grams = generate_n_gram(sentence, max_n_gram=3)
  novelty = sum([novelty_score(new_sentence_n_grams, past_sentences[i]) for i in past_sentences])/len(past_sentences)
  return novelty

In [ ]:
past_sentences[1] = generate_n_gram("hello there general kenobi", 3)
past_sentences[2] = generate_n_gram("hello there foo bar foobar",3)

In [ ]:
past_sentences[1]

[['hello', 'there', 'general', 'kenobi'],
 ['hello there', 'there general', 'general kenobi'],
 ['hello there general', 'there general kenobi']]

In [ ]:
calculate_novelty("hello there general kenobi")

0.6388888888888888

In [ ]:
predictions = ["hello there general kenobi", "hello there foo bar foobar"]
s1 = generate_N_grams(predictions[0],2)
s2 = generate_N_grams(predictions[1],2)
unique = set(s1).intersection(set(s2))
print(len(unique)/len(s1))

Sentence after removing stopwords: ['hello', 'there', 'general', 'kenobi']
Sentence after removing stopwords: ['hello', 'there', 'foo', 'bar', 'foobar']
0.3333333333333333


# Function to select best utterance

In [ ]:
from torch.utils.data.dataset import Dataset
def get_dataset(emotion):
  if emotion == "sad":
    dataset = sad_dialogue
  return dataset

In [ ]:
def get_response(emotion, situation):
  for 

In [ ]:
def score(row):
  score = row["ppl_how_recent_is_the_event?"]
  return {'score': score}

In [ ]:
top_10_similar.map(score)

  0%|          | 0/11 [00:00<?, ?ex/s]

Dataset({
    features: ['age', 'sad_utterance', 'how_recent_is_the_event?', 'specific_event_trigger', 'additional_questions_permission', 'feeling_envy_mistrust_trigger', 'method_6', 'self_blame_trigger', 'gender', 'embeddings', 'acd', 'ppl_how_recent_is_the_event?', 'ppl_specific_event_trigger', 'ppl_additional_questions_permission', 'ppl_feeling_envy_mistrust_trigger', 'ppl_method_6', 'similarity', 'score'],
    num_rows: 11
})

In [ ]:
# choosing a random sentence
top_10.sample(n=3)['specific_event_trigger']

44    Вижу, что вам не очень хорошо, спасибо вам за ...
48    Мне очень жаль, что вам так плохо. Ваша грусть...
68    Мне жаль, что у тебя плохое настроение. Думаю,...
Name: specific_event_trigger, dtype: object

## Fluency

In [ ]:
corpus_embeddings = epru['train']['embeddings'].to('cuda')
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

# query_embeddings = query_embeddings.to('cuda')
# query_embeddings = util.normalize_embeddings(query_embeddings)
# hits = util.semantic_search(query_embeddings, corpus_embeddings, score_function=util.dot_score)

AttributeError: ignored

In [ ]:
epru.set_format('torch', columns=['embeddings'])

In [ ]:
corpus_embeddings = epru['train']['embeddings'][0][0]
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)


AttributeError: ignored

In [ ]:
query = "рудг"
query_embedding = pipe(query)
querry_embedding[0][0].shape

AttributeError: ignored

In [ ]:
util.cos_sim(query_embedding[0][0], epru['train']['embeddings'][100][0][0])

tensor([[0.3523]])

In [ ]:
epru['train']['embeddings'][100][0][0].shape

(768,)